In [34]:
import numpy as np
from sklearn.metrics import roc_auc_score
def STL_AUC(model,SDG,y_true):
    Skip_Tag = []
        
    y_pred = model.predict_generator(generator = SDG,
                                        workers = 12,
                                        use_multiprocessing=True,) # (#data x 50())  # (#data x 50()) 
    y_pred_out = y_pred.copy()
    #print(y_pred.shape,self.y.shape,len(self.SDG.text_filenames))
    for i in range(len(y_pred)):
        for j in range(len(y_pred[i])):
            if y_pred[i][j] >= 0.5 : 
                y_pred[i][j] = 1
            else:
                y_pred[i][j] = 0
    
    #print(y_pred.shape,y_true.shape)
    
    try:
        AUC_this_epoch = roc_auc_score(y_true ,y_pred)
    except:
        AUC_this_epoch = 0
    
    
    AUC_List = []
    y_pred_T = y_pred.T
    y_true_T = y_true.T
    
    for i in range(50):
        
        try:
            AUC_List.append( roc_auc_score(y_true_T[i] ,y_pred_T[i]))
        except:
            AUC_List.append(0)
    return AUC_this_epoch,AUC_List,y_pred_out

In [2]:
##read Data Base
from pymongo import MongoClient
from bson.objectid import ObjectId
uri = "mongodb://localhost:27017/database" #mongodb://<user_name>:<user_password>@ds<xxxxxx>.mlab.com:<xxxxx>/<database_name>
conn =  MongoClient(host='localhost', port=27017, connect=False)
db = conn.PaperData
Spectrogram_Collection  =  db.get_collection('MSD_mel1366')
#LineCNN_Collection  =  db.get_collection('LineCNNMatrix_100thTag')

In [3]:
Spectrogram_Collection.count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


58487

In [4]:
Spectrogram_Collection.create_index([("Filename","text")])

'Filename_text'

In [5]:
#Audio Para
_mel_scale = 96
_time_len = 1366
_channels = 1

In [6]:
_num_Tags = 50
_epchos = 200
_stopEpcho = 10
_batchSize = 32

In [7]:
import json 
import codecs
import numpy as np
def read_json(filename):
    with codecs.open(filename,'r',encoding = 'utf8') as infile:
        return np.array(json.load(infile))
    
x_train = read_json('New_x_train.json')
y_train = read_json('Old_y_train.json')
x_test = read_json('New_x_test.json')
y_test = read_json('Old_y_test.json')
x_val = read_json('New_x_val.json')
y_val = read_json('Old_y_val.json')

In [8]:
print(len(x_train) == len(y_train))
print(len(x_test) == len(y_test))
print(len(x_val) == len(y_val))

True
True
True


In [9]:
from keras.layers import *
from keras.models import Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical

Using TensorFlow backend.


In [10]:
_CL_1_kernal = 169
_CL_2_kernal = 339
_CL_3_kernal = 339
_CL_4_kernal = 339
_RNN_1_kernal = 169
_RNN_2_kernal = 169


In [11]:
Spectrogram_Input = Input(name= 'AudioInput',shape=(_mel_scale, _time_len, _channels)) #(None(1), 96, 1292, 1)
CL_1 = Conv2D(_CL_1_kernal, kernel_size=(3, 3),padding='same', name = "CL_1")(Spectrogram_Input) #(None, 96, 1292, 128) 
BN_1 = BatchNormalization()(CL_1)
AL_1 = ELU()(BN_1)
MP_1 = MaxPool2D(pool_size= (2,2), name = "MPL_1")(AL_1) #(None, 48, 323, 128)
DP_1 = Dropout(0.1)(MP_1)

CL_2 = Conv2D(_CL_2_kernal, kernel_size=(3, 3),padding='same', name = "CL_2")(DP_1) # (None, 48, 323, 384) 
BN_2 = BatchNormalization()(CL_2)
AL_2 = ELU()(BN_2)
MP_2 = MaxPool2D(pool_size= (3,3), name = "MPL_2")(AL_2) # (None, 12, 64, 384) 
DP_2 = Dropout(0.1)(MP_2)

CL_3 = Conv2D(_CL_3_kernal, kernel_size=(3, 3),padding='same', name = "CL_3")(DP_2) #(None, 12, 64, 768) 
BN_3 = BatchNormalization()(CL_3)
AL_3 = ELU()(BN_3)
MP_3 = MaxPool2D(pool_size= (4,4), name = "MPL_3")(AL_3) #(None, 4, 8, 768)
DP_3 = Dropout(0.1)(MP_3)

CL_4 = Conv2D(_CL_4_kernal, kernel_size=(3, 3),padding='same', name = "CL_4")(DP_3) #(None, 4, 8, 2048) 
BN_4 = BatchNormalization()(CL_4)
AL_4 = ELU()(BN_4)
MP_4 = MaxPool2D(pool_size= (4,4), name = "MPL_4")(AL_4)#(None, 1, 1, 2048) 
DP_4 = Dropout(0.1)(MP_4)

AR = Permute((3,2,1))(DP_4)
RS = Reshape((14, _CL_2_kernal))(DP_4)

GRU_1 = GRU(_RNN_1_kernal, return_sequences=True, name='GRU_1')(RS)
GRU_2 = GRU(_RNN_2_kernal, return_sequences=False, name='GRU_2')(GRU_1)

#pseudo-model for sheck summary
Softmax = Dense(_num_Tags, kernel_initializer='normal',activation='sigmoid')(GRU_2)
Audio_model = Model(inputs=Spectrogram_Input, outputs=Softmax)

In [12]:
Audio_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
AudioInput (InputLayer)      (None, 96, 1366, 1)       0         
_________________________________________________________________
CL_1 (Conv2D)                (None, 96, 1366, 169)     1690      
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1366, 169)     676       
_________________________________________________________________
elu_1 (ELU)                  (None, 96, 1366, 169)     0         
_________________________________________________________________
MPL_1 (MaxPooling2D)         (None, 48, 683, 169)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 683, 169)      0         
_________________________________________________________________
CL_2 (Conv2D)                (None, 48, 683, 339)      515958    
__________

In [13]:
from keras.utils import Sequence
import json
import codecs
class AudioDataGenerator(Sequence):
    #batch size can only be 1
    def __init__(self, text_filenames, labels,batch_size):
        self.text_filenames, self.labels = text_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.text_filenames)/ (self.batch_size)))

    def __getitem__(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        filename_List =  self.text_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        #Text_batch_x = [] 
        Audio_batch_x = []
        for filename in filename_List:  
            Audio_x = np.array(Spectrogram_Collection.find_one({"Filename":filename})['Spectrogram']).reshape((_mel_scale,_time_len,_channels))
            Audio_batch_x.append(Audio_x)
        y_List = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return np.array(Audio_batch_x), np.array(y_List)
    def getitem(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        
        return self.__getitem__(idx)

In [14]:
ADG_train = AudioDataGenerator(x_train, y_train, _batchSize)
ADG_test = AudioDataGenerator(x_test[:8480], y_test[:8480], _batchSize) #skip 6 song while training
ADG_val = AudioDataGenerator(x_val, y_val, _batchSize)

In [15]:
## This callback is for binary softmax 

from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class AUC_callback(Callback):
    def __init__(self,y_true,SDG,batchsize):
        self.y_true = y_true
        self.SDG = SDG
        self.step = (len(y_true) // _batchSize)
        
    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        
        Skip_Tag = []
        
        y_pred = self.model.predict_generator(generator = self.SDG,
                                         steps = self.step,
                                        workers = 12,
                                        use_multiprocessing=True,) # (#data x 50()) 
        y_pred_out = y_pred.copy()
    #print(y_pred.shape,self.y.shape,len(self.SDG.text_filenames))
        for i in range(len(y_pred)):
            for j in range(len(y_pred[i])):
                if y_pred[i][j] >= 0.5 : 
                    y_pred[i][j] = 1
                else:
                    y_pred[i][j] = 0
        try:
            AUC_this_epoch = roc_auc_score(self.y_true ,y_pred)
        except Exception as e:
            AUC_this_epoch = 0
            print(e)
    
        AUC_List = []
        y_pred_T = y_pred.T
        y_true_T = self.y_true .T
    
        for i in range(50):
            try:
                AUC_List.append( roc_auc_score(y_true_T[i] ,y_pred_T[i]))
            except:
                AUC_List.append(0)

        logs['AUC_test'] = AUC_this_epoch
        
        logs['AUC_List'] = AUC_List
        
        print('AUC_test: %s '% (str(round(AUC_this_epoch,5))))
        
        
        
    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [16]:
## This callback is for binary softmax 

from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class AUC_callback_TF(Callback):
    def __init__(self,y_true,SDG,batchsize):
        self.y_true = y_true
        self.SDG = SDG
        self.step = (len(y_true) // _batchSize)
        
    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        
        Skip_Tag = []
        print("[AUC] Start predict..." ,end = '  ')
        y_pred = self.model.predict_generator(generator = self.SDG,
                                         steps = self.step,
                                        workers = 12,
                                        use_multiprocessing=True,) # (#data x 50()) 
        y_pred_out = y_pred.copy()
        print("[OK]")
    #print(y_pred.shape,self.y.shape,len(self.SDG.text_filenames))
        print("[AUC] Start calcute auc...", end = '  ')
        for i in range(len(y_pred)):
            for j in range(len(y_pred[i])):
                if y_pred[i][j] >= 0.5 : 
                    y_pred[i][j] = 1
                else:
                    y_pred[i][j] = 0
        try:
            auc, update_op = tf.metrics.auc(self.y_true ,K.round(y_pred))
            with tf.Session() as sess:
                sess.run(tf.local_variables_initializer())
                AUC_this_epoch= sess.run([auc, update_op])[0]
        except Exception as e:
            AUC_this_epoch = 0
            print(e)
        print("[Total OK!]")
        
        #AUC_List = []
        #y_pred_T = y_pred.T
        #y_true_T = self.y_true.T
    
        #for i in range(50):
        #    try:
        #        auc, update_op = tf.metrics.auc(y_true_T[i] ,K.round(y_pred_T[i]))
        #        with tf.Session() as sess:
        #            sess.run(tf.local_variables_initializer())
        #            AUC_List.append(sess.run([auc, update_op])[0])
        #    except Exception as e:
        #        AUC_List.append(0)
        #        print(e)
        #print("[Tag OK!]")
        logs['AUC_test'] = AUC_this_epoch
        #logs['AUC_List'] = AUC_List
        print('AUC_test: %s '% (str(round(AUC_this_epoch,5))))
        
        
    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [17]:
Test_AUC = AUC_callback_TF(y_test[:8480],ADG_test,_batchSize)

In [18]:
import tensorflow as tf
auc, update_op = tf.metrics.auc(np.array([[0,0,1],[1,0,1],[0,1,0]]), np.array([[0,0,0],[0,0,0],[0,0,0]]))
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("tf auc: {}".format(sess.run([auc, update_op])))

tf auc: [0.5, 0.5]


In [19]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='AUC_test',mode='max', patience=_stopEpcho, verbose=1)

In [20]:
from keras.callbacks import Callback
import time
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.epoch_time_end = time.time()
        self.times.append(self.epoch_time_end - self.epoch_time_start)
        logs['Timer'] = self.epoch_time_end - self.epoch_time_start
time_callback = TimeHistory()


In [21]:
import keras_metrics
precision = keras_metrics.precision(label = 1)
recall = keras_metrics.recall(label = 0)

In [22]:
#https://www.quora.com/Why-does-my-convolutional-neural-network-always-produce-the-same-outputs

In [23]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath="TVT_Audio_CRNN_Old_best.hdf5",monitor='AUC_test',mode = 'max', verbose=1, save_best_only=True, )

In [24]:
from keras import backend as K
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true,  K.round(y_pred))[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [25]:
num_training_samples = len(x_train)
num_validation_samples = len(x_val)
num_test_samples =  len(x_test)

In [26]:
#THis may be the final version of CRNN audio !!!
import warnings
warnings.filterwarnings('ignore')
from keras.utils import multi_gpu_model
parallel_model = multi_gpu_model(Audio_model, gpus=2)
parallel_model.compile(loss="binary_crossentropy",
                      optimizer='adam',
                      metrics=[auc])
History = parallel_model.fit_generator(
                    generator= ADG_train,
                    steps_per_epoch=(num_training_samples // _batchSize),
                    epochs= 100,
                    verbose=1,
                    validation_data= ADG_val,
                    validation_steps=(num_validation_samples // _batchSize),
                    workers=12, use_multiprocessing=True,
                    callbacks = [Test_AUC,early_stopping,time_callback, checkpointer]
                    )

Epoch 1/100
928/928 [==============================] - 541s 583ms/step - loss: 0.2953 - auc: 0.5101 - val_loss: 0.3005 - val_auc: 0.5184
[AUC] Start predict...  [OK]
[AUC] Start calcute auc...  [Total OK!]
AUC_test: 0.52453 

Epoch 00001: AUC_test improved from -inf to 0.52453, saving model to TVT_Audio_CRNN_best.hdf5
Epoch 2/100
928/928 [==============================] - 535s 576ms/step - loss: 0.2826 - auc: 0.5206 - val_loss: 0.3222 - val_auc: 0.5221
[AUC] Start predict...  [OK]
[AUC] Start calcute auc...  [Total OK!]
AUC_test: 0.51665 

Epoch 00002: AUC_test did not improve from 0.52453
Epoch 3/100
928/928 [==============================] - 536s 578ms/step - loss: 0.2746 - auc: 0.5242 - val_loss: 0.2959 - val_auc: 0.5264
[AUC] Start predict...  [OK]
[AUC] Start calcute auc...  [Total OK!]
AUC_test: 0.52511 

Epoch 00003: AUC_test improved from 0.52453 to 0.52511, saving model to TVT_Audio_CRNN_best.hdf5
Epoch 4/100
928/928 [==============================] - 536s 578ms/step - loss: 0

928/928 [==============================] - 538s 579ms/step - loss: 0.1871 - auc: 0.6093 - val_loss: 0.2957 - val_auc: 0.6109
[AUC] Start predict...  [OK]
[AUC] Start calcute auc...  [Total OK!]
AUC_test: 0.59887 

Epoch 00029: AUC_test did not improve from 0.61850
Epoch 30/100
928/928 [==============================] - 534s 576ms/step - loss: 0.1838 - auc: 0.6125 - val_loss: 0.2919 - val_auc: 0.6140
[AUC] Start predict...  [OK]
[AUC] Start calcute auc...  [Total OK!]
AUC_test: 0.61161 

Epoch 00030: AUC_test did not improve from 0.61850
Epoch 31/100
928/928 [==============================] - 538s 580ms/step - loss: 0.1808 - auc: 0.6156 - val_loss: 0.3019 - val_auc: 0.6171
[AUC] Start predict...  [OK]
[AUC] Start calcute auc...  [Total OK!]
AUC_test: 0.59126 

Epoch 00031: AUC_test did not improve from 0.61850
Epoch 32/100
928/928 [==============================] - 538s 579ms/step - loss: 0.1782 - auc: 0.6187 - val_loss: 0.3068 - val_auc: 0.6202
[AUC] Start predict...  [OK]
[AUC] Start 

In [28]:
for i in range(len(History.history['AUC_test'])):
    History.history['AUC_test'][i] = History.history['AUC_test'][i].item()

In [29]:
from keras.models import model_from_json
json_string = parallel_model.to_json()
with codecs.open('Audio_CRNN_old.json','w', encoding = 'utf8') as outfile:
    json.dump(json_string,outfile)
with codecs.open('Audio_CRNN_old_History.json','w', encoding = 'utf8') as outfile:
    json.dump(History.history,outfile)

In [30]:
parallel_model.load_weights('TVT_Audio_CRNN_Old_best.hdf5')

In [38]:
y_pred = parallel_model.predict_generator(ADG_test,workers=12,use_multiprocessing=True)

In [39]:
y_pred.shape

(8480, 50)

In [40]:
y_pred_out = y_pred.copy()
#print(y_pred.shape,self.y.shape,len(self.SDG.text_filenames))
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if y_pred[i][j] >= 0.5 : 
            y_pred[i][j] = 1
        else:
            y_pred[i][j] = 0

In [44]:
y_test[8480].shape

(50,)

In [45]:
try:
    auc, update_op = tf.metrics.auc(y_test[:8480] ,K.round(y_pred))
    with tf.Session() as sess:
        sess.run(tf.local_variables_initializer())
        print("tf auc: {}".format(sess.run([auc, update_op])))
except Exception as e:
    AUC_this_epoch = 0
    print(e)

tf auc: [0.61850405, 0.61850405]


In [48]:
AUC_List = []
y_pred_T = y_pred.T
y_test_T = y_test[:8480].T
    
for i in range(50):
        
    try:
        auc, update_op = tf.metrics.auc(y_test_T[i] ,K.round(y_pred_T[i]))
        with tf.Session() as sess:
            sess.run(tf.local_variables_initializer())
            print("tf auc: {}".format(sess.run([auc, update_op])))
        AUC_List.append(auc)
        
    except:
        AUC_List.append(0)

tf auc: [0.6438254, 0.6438254]
tf auc: [0.63034207, 0.63034207]
tf auc: [0.6140687, 0.6140687]
tf auc: [0.73055744, 0.73055744]
tf auc: [0.6595194, 0.6595194]
tf auc: [0.73368037, 0.73368037]
tf auc: [0.5192147, 0.5192147]
tf auc: [0.5356427, 0.5356427]
tf auc: [0.6223979, 0.6223979]
tf auc: [0.5321041, 0.5321041]
tf auc: [0.5726798, 0.5726798]
tf auc: [0.53070086, 0.53070086]
tf auc: [0.56584775, 0.56584775]
tf auc: [0.6273218, 0.6273218]
tf auc: [0.7119799, 0.7119799]
tf auc: [0.55210257, 0.55210257]
tf auc: [0.5234359, 0.5234359]
tf auc: [0.50505906, 0.50505906]
tf auc: [0.5936634, 0.5936634]
tf auc: [0.61326844, 0.61326844]
tf auc: [0.6915113, 0.6915113]
tf auc: [0.616504, 0.616504]
tf auc: [0.57522607, 0.57522607]
tf auc: [0.6490817, 0.6490817]
tf auc: [0.55495, 0.55495]
tf auc: [0.50427324, 0.50427324]
tf auc: [0.5013928, 0.5013928]
tf auc: [0.53734064, 0.53734064]
tf auc: [0.5053851, 0.5053851]
tf auc: [0.5, 0.5]
tf auc: [0.71312386, 0.71312386]
tf auc: [0.6096273, 0.6096273]
tf